In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np # Often useful

print("Libraries imported.")

# Load environment variables from .env file
print("Loading .env variables...")
# Use override=True if you might change .env and restart kernel,
# otherwise it might not pick up changes easily.
load_dotenv(override=True)
kg_git_dir = os.environ.get('KG_GIT')

# Check if KG_GIT path was loaded correctly
if not kg_git_dir or not os.path.isdir(kg_git_dir):
   raise ValueError(f"KG_GIT path not found or not set correctly in .env: Check the path and ensure it ends with '/'")
print(f"Project directory read from .env: {kg_git_dir}")
print(".env loaded.")

Libraries imported.
Loading .env variables...
Project directory read from .env: /Users/gastondana/Desktop/spoke_genelab/
.env loaded.


In [3]:
# --- Define Input File Paths ---
# !! Adjust download_dir if you saved the files elsewhere !!
download_dir = os.path.join(kg_git_dir, 'downloaded_data') # Assumes you created this subfolder
sample_table_file = os.path.join(kg_git_dir, 'OSD-679_SampleTable.csv')
# ADJUST THESE TWO LINES:
oct_data_file = os.path.join(kg_git_dir, 'LSDS-81_Ophthalmologic Diagnostic Technique_Fuller_OCT_TRANSFORMED.csv')
data_dict_file = os.path.join(kg_git_dir, 'OSD-679_sup_Fuller_DataDictionary.xlsx')

print(f"Sample Table Path: {sample_table_file}")
print(f"OCT Data Path: {oct_data_file}")
print(f"Data Dictionary Path: {data_dict_file}")

# --- Check if files exist (optional but good) ---
if not os.path.exists(sample_table_file): print(f"\nWARNING: Sample table file not found at:\n{sample_table_file}")
if not os.path.exists(oct_data_file): print(f"\nWARNING: OCT data file not found at:\n{oct_data_file}")
if not os.path.exists(data_dict_file): print(f"\nWARNING: Data dictionary file not found at:\n{data_dict_file}")

Sample Table Path: /Users/gastondana/Desktop/spoke_genelab/OSD-679_SampleTable.csv
OCT Data Path: /Users/gastondana/Desktop/spoke_genelab/LSDS-81_Ophthalmologic Diagnostic Technique_Fuller_OCT_TRANSFORMED.csv
Data Dictionary Path: /Users/gastondana/Desktop/spoke_genelab/OSD-679_sup_Fuller_DataDictionary.xlsx


In [4]:
# --- Load Input Data ---
print("Loading Sample Table CSV...")
try:
    sample_df = pd.read_csv(sample_table_file)
    print(f"Loaded Sample Table: {sample_df.shape[0]} rows, {sample_df.shape[1]} columns")
    display(sample_df.head(3)) # Show first 3 rows
except Exception as e:
    print(f"ERROR loading Sample Table CSV: {e}")
    sample_df = None # Set to None if loading failed

print("\nLoading OCT Data CSV...")
try:
    oct_df = pd.read_csv(oct_data_file)
    print(f"Loaded OCT Data: {oct_df.shape[0]} rows, {oct_df.shape[1]} columns")
    display(oct_df.head(3)) # Show first 3 rows
except Exception as e:
    print(f"ERROR loading OCT Data CSV: {e}")
    oct_df = None # Set to None if loading failed

Loading Sample Table CSV...
Loaded Sample Table: 644 rows, 2 columns


,Sample Name,Treatment Group
0,FT01_OD,3 month & Ambient Air & Not Applicable & Norma...
1,FT01_OS,3 month & Ambient Air & Not Applicable & Norma...
2,FT02_OD,3 month & Ambient Air & Not Applicable & Hindl...



Loading OCT Data CSV...
Loaded OCT Data: 599 rows, 24 columns


,Sample Name,Baseline _Nasal_Total_Retinal_Thickness_millimeter,Baseline _Nasal_Choroidal_Thickness_millimeter,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_Thickness_millimeter,End_of_Hindlimb_Unloading_Nasal_Choroidal_Thickness_millimeter,End_of_Recovery_Nasal_Total_Retinal_Thickness_millimeter,End_of_Recovery_Nasal_Choroidal_Thickness_millimeter,Baseline _Temporal_Total_Retinal_Thickness_millimeter,Baseline _Temporal_Choroidal_Thickness_millimeter,End_of_Hindlimb_Unloading_Temporal_Total_Retinal_Thickness_millimeter,...,Baseline _Linear_Choroidal_Thickness_millimeter,End_of_Hindlimb_Unloading_Linear_Total_Retinal_Thickness_millimeter,End_of_Hindlimb_Unloading_Linear_Choroidal_Thickness_millimeter,End_of_Recovery_Linear_Total_Retinal_Thickness_millimeter,End_of_Recovery_Linear_Choroidal_Thickness_millimeter,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,MT305_OD,0.185,0.052,NaN,NaN,NaN,NaN,0.176,0.055,NaN,...,0.056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MT306_OD,0.194,0.059,NaN,NaN,NaN,NaN,0.189,0.059,NaN,...,0.061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MT307_OD,0.194,0.053,NaN,NaN,NaN,NaN,0.192,0.057,NaN,...,0.052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# --- Reshape OCT Data from Wide to Long Format ---
print("Reshaping OCT data...")
if oct_df is not None: # Check if oct_df loaded correctly in the previous cell
    try:
        # Identify the ID column that should remain a column
        id_vars = ['Sample Name']

        # Identify all the measurement columns (all columns *except* Sample Name)
        value_vars = [col for col in oct_df.columns if col not in id_vars]

        # Perform the melt operation
        oct_df_long = pd.melt(
            oct_df,
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='MeasurementColumnHeader', # New column holding the original header
            value_name='MeasurementValue'      # New column holding the measurement number
        )

        # Remove rows where the measurement value is missing (NaN)
        # These rows don't represent actual measurements
        print(f"Original number of measurement entries: {len(oct_df_long)}")
        oct_df_long.dropna(subset=['MeasurementValue'], inplace=True)
        print(f"Number of actual measurement entries after dropping NAs: {len(oct_df_long)}")

        # Reset index after dropping rows
        oct_df_long.reset_index(drop=True, inplace=True)

        print(f"Reshaped data into long format: {oct_df_long.shape[0]} rows, {oct_df_long.shape[1]} columns")
        print("Example rows of long format data:")
        display(oct_df_long.head()) # Show first 5 rows of the new long DataFrame

    except Exception as e:
        print(f"ERROR reshaping data: {e}")
        oct_df_long = None # Set to None if reshaping failed
else:
    print("Skipping reshape because oct_df did not load correctly.")
    oct_df_long = None

Reshaping OCT data...
Original number of measurement entries: 13777
Number of actual measurement entries after dropping NAs: 6984
Reshaped data into long format: 6984 rows, 3 columns
Example rows of long format data:


,Sample Name,MeasurementColumnHeader,MeasurementValue
0,MT305_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.185
1,MT306_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.194
2,MT307_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.194
3,MT308_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.204
4,MT309_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.198


In [6]:
# --- Parse Measurement Column Header ---
print("Parsing MeasurementColumnHeader...")
if oct_df_long is not None: # Check if oct_df_long exists from previous cell
    try:
        # Define a regular expression pattern to capture the parts:
        # Pattern breakdown:
        # ^ = Start of string
        # (Baseline|End_of_Hindlimb_Unloading|End_of_Recovery) = Capture Group 1: Timepoint
        # _ = Underscore delimiter
        # (Nasal|Temporal|Linear) = Capture Group 2: Location
        # _ = Underscore delimiter
        # (.*) = Capture Group 3: Measurement Type (everything else until the unit)
        # _millimeter = The unit part
        # $ = End of string
        pattern = r"^(Baseline|End_of_Hindlimb_Unloading|End_of_Recovery)_(Nasal|Temporal|Linear)_(.*)_millimeter$"

        # Use .str.extract() to apply the pattern and create new columns from capture groups
        extracted_data = oct_df_long['MeasurementColumnHeader'].str.extract(pattern)

        # Assign the extracted parts to new columns in the DataFrame
        oct_df_long['Timepoint'] = extracted_data[0]
        oct_df_long['Location'] = extracted_data[1]
        oct_df_long['MeasurementType'] = extracted_data[2]

        # Clean up the MeasurementType column by replacing underscores with spaces
        oct_df_long['MeasurementType'] = oct_df_long['MeasurementType'].str.replace('_', ' ')

        # Add the Unit column explicitly since we know it from the pattern
        oct_df_long['Unit'] = 'millimeter'

        print("Successfully parsed header components.")
        print("DataFrame with new columns (first 5 rows):")
        display(oct_df_long.head()) # Show the DataFrame with the new columns

        # Optional: Check if any headers didn't match the pattern
        failed_parse_count = oct_df_long['Timepoint'].isna().sum()
        if failed_parse_count > 0:
            print(f"\nWARNING: {failed_parse_count} rows might not have parsed correctly (check for NaNs).")
        else:
            print("\nParsing check passed (no obvious failures).")

    except Exception as e:
        print(f"ERROR parsing header column: {e}")
        # You might want to stop execution here if parsing fails critically
else:
    print("Skipping parsing because oct_df_long is not available.")

Parsing MeasurementColumnHeader...
Successfully parsed header components.
DataFrame with new columns (first 5 rows):


,Sample Name,MeasurementColumnHeader,MeasurementValue,Timepoint,Location,MeasurementType,Unit
0,MT305_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.185,NaN,NaN,NaN,millimeter
1,MT306_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.194,NaN,NaN,NaN,millimeter
2,MT307_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.194,NaN,NaN,NaN,millimeter
3,MT308_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.204,NaN,NaN,NaN,millimeter
4,MT309_OD,Baseline _Nasal_Total_Retinal_Thickness_millim...,0.198,NaN,NaN,NaN,millimeter


In [7]:
# --- Filter out rows where parsing failed ---
print("Filtering out rows that didn't match the expected header pattern...")
if oct_df_long is not None: # Check if the dataframe exists
    original_rows = len(oct_df_long)

    # Keep only rows where 'Timepoint' is not NaN (meaning the regex pattern successfully matched)
    oct_df_long_parsed = oct_df_long.dropna(subset=['Timepoint']).copy()

    # Reset the index of the new dataframe
    oct_df_long_parsed.reset_index(drop=True, inplace=True)

    removed_rows = original_rows - len(oct_df_long_parsed)
    print(f"Removed {removed_rows} rows due to parsing failure (likely from unnamed columns).")
    print(f"DataFrame now contains {len(oct_df_long_parsed)} rows with successfully parsed data.")
    print("Example rows of successfully parsed data:")
    display(oct_df_long_parsed.head()) # Display the cleaned dataframe
else:
    print("Skipping filtering because oct_df_long is not available.")
    oct_df_long_parsed = None # Make sure variable exists even if filtering skipped

Filtering out rows that didn't match the expected header pattern...
Removed 2989 rows due to parsing failure (likely from unnamed columns).
DataFrame now contains 3995 rows with successfully parsed data.
Example rows of successfully parsed data:


,Sample Name,MeasurementColumnHeader,MeasurementValue,Timepoint,Location,MeasurementType,Unit
0,MT51_OD,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.228,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter
1,MT56_OD,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.241,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter
2,MT59_OD,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.211,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter
3,MT51_OS,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.22,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter
4,MT56_OS,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.224,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter


In [8]:
# --- Merge Sample Group Information ---
print("Merging sample treatment group information...")
# Check if both dataframes needed for merge exist from previous steps
if 'oct_df_long_parsed' in locals() and oct_df_long_parsed is not None and \
   'sample_df' in locals() and sample_df is not None:
    try:
        # Just in case, select only the columns we need from sample_df
        sample_groups = sample_df[['Sample Name', 'Treatment Group']].copy()
        # Ensure no duplicate Sample Names exist in the sample groups table
        sample_groups = sample_groups.drop_duplicates(subset=['Sample Name'])

        # Perform a 'left' merge: keeps all rows from oct_df_long_parsed
        # and adds 'Treatment Group' where 'Sample Name' matches.
        oct_data_with_groups = pd.merge(
            oct_df_long_parsed,
            sample_groups,
            on='Sample Name', # The column to match between the two tables
            how='left'
        )

        print(f"Merge complete. DataFrame shape is now: {oct_data_with_groups.shape}")
        print("Example rows with Treatment Group added:")
        display(oct_data_with_groups.head()) # Display first few rows of the merged data

        # Optional: Check if any measurement rows failed to find a matching sample group
        failed_merge_count = oct_data_with_groups['Treatment Group'].isna().sum()
        if failed_merge_count > 0:
            print(f"\nWARNING: {failed_merge_count} measurement rows could not be matched with a treatment group in the sample table.")
        else:
            print("\nMerge check passed (all measurement rows matched a sample group).")

    except Exception as e:
        print(f"ERROR merging dataframes: {e}")
        oct_data_with_groups = None # Indicate failure
else:
    print("Skipping merge because prerequisite DataFrames (oct_df_long_parsed or sample_df) are not available.")
    oct_data_with_groups = None

Merging sample treatment group information...
Merge complete. DataFrame shape is now: (3995, 8)
Example rows with Treatment Group added:


,Sample Name,MeasurementColumnHeader,MeasurementValue,Timepoint,Location,MeasurementType,Unit,Treatment Group
0,MT51_OD,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.228,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter,3 month & Ambient Air & Not Applicable & Hindl...
1,MT56_OD,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.241,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter,3 month & Ambient Air & Not Applicable & Hindl...
2,MT59_OD,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.211,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter,3 month & Ambient Air & Not Applicable & Hindl...
3,MT51_OS,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.22,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter,3 month & Ambient Air & Not Applicable & Hindl...
4,MT56_OS,End_of_Hindlimb_Unloading_Nasal_Total_Retinal_...,0.224,End_of_Hindlimb_Unloading,Nasal,Total Retinal Thickness,millimeter,3 month & Ambient Air & Not Applicable & Hindl...



Merge check passed (all measurement rows matched a sample group).


In [9]:
# --- Map Anatomical Locations to UBERON IDs ---
print("Mapping locations to UBERON IDs...")
# Check if the DataFrame exists from the previous step
if 'oct_data_with_groups' in locals() and oct_data_with_groups is not None:
    try:
        # Define the mapping from the Location column string to the corresponding UBERON ID
        # Based on our BioPortal lookups and the Data Dictionary for 'Linear'
        location_to_uberon = {
            'Nasal': 'UBERON:0016823',    # Nasal part of retina
            'Temporal': 'UBERON:0016824', # Temporal part of retina
            'Linear': 'UBERON:0001773'     # Optic Disc (aka Optic Nerve Head)
        }

        # Create a new column 'AnatomyID' by applying this mapping to the 'Location' column
        oct_data_with_groups['AnatomyID'] = oct_data_with_groups['Location'].map(location_to_uberon)

        print("Anatomy IDs mapped.")
        print("DataFrame with AnatomyID column added (showing relevant columns):")
        # Display head showing key ID/location columns and the measurement value
        display(oct_data_with_groups[[
            'Sample Name',
            'Timepoint',
            'Location',
            'AnatomyID', # The new column
            'MeasurementType',
            'MeasurementValue'
        ]].head())

        # Optional: Check if any locations in the data didn't match our map
        failed_map_count = oct_data_with_groups['AnatomyID'].isna().sum()
        if failed_map_count > 0:
            print(f"\nWARNING: {failed_map_count} rows had a location in the 'Location' column that wasn't 'Nasal', 'Temporal', or 'Linear'.")
            print("Unique locations found in data:", oct_data_with_groups['Location'].unique())
        else:
            print("\nAnatomy ID mapping check passed (all locations mapped).")

    except Exception as e:
        print(f"ERROR mapping anatomy terms: {e}")
else:
    print("Skipping anatomy mapping because oct_data_with_groups DataFrame is not available.")

Mapping locations to UBERON IDs...
Anatomy IDs mapped.
DataFrame with AnatomyID column added (showing relevant columns):


,Sample Name,Timepoint,Location,AnatomyID,MeasurementType,MeasurementValue
0,MT51_OD,End_of_Hindlimb_Unloading,Nasal,UBERON:0016823,Total Retinal Thickness,0.228
1,MT56_OD,End_of_Hindlimb_Unloading,Nasal,UBERON:0016823,Total Retinal Thickness,0.241
2,MT59_OD,End_of_Hindlimb_Unloading,Nasal,UBERON:0016823,Total Retinal Thickness,0.211
3,MT51_OS,End_of_Hindlimb_Unloading,Nasal,UBERON:0016823,Total Retinal Thickness,0.22
4,MT56_OS,End_of_Hindlimb_Unloading,Nasal,UBERON:0016823,Total Retinal Thickness,0.224



Anatomy ID mapping check passed (all locations mapped).


In [10]:
# --- Generate/Extract Unique IDs ---
print("Generating unique identifiers...")
# Check if the DataFrame exists from the previous step
if 'oct_data_with_groups' in locals() and oct_data_with_groups is not None:
    try:
        # 1. Subject ID: Extract the base animal ID from 'Sample Name'
        #    (Removes the '_OD' or '_OS' suffix)
        oct_data_with_groups['SubjectID'] = oct_data_with_groups['Sample Name'].str.replace(r'_OD|_OS', '', regex=True)
        print(f"Extracted Subject IDs (e.g., '{oct_data_with_groups['SubjectID'].iloc[0]}').")

        # 2. Assay ID: Create a unique ID for each specific assay instance
        #    (Combination of dataset, tech, sample, eye, timepoint)
        oct_data_with_groups['AssayID'] = 'OSD-679_OCT_' + \
                                          oct_data_with_groups['Sample Name'] + '_' + \
                                          oct_data_with_groups['Timepoint']
        print(f"Generated Assay IDs (e.g., '{oct_data_with_groups['AssayID'].iloc[0]}').")

        # 3. Measurement ID: Create a unique ID for each measurement value row
        #    (Based on AssayID + Location + MeasurementType, making type safe for ID)
        safe_measurement_type = oct_data_with_groups['MeasurementType'].str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
        oct_data_with_groups['MeasurementID'] = oct_data_with_groups['AssayID'] + '_' + \
                                                  oct_data_with_groups['Location'] + '_' + \
                                                  safe_measurement_type
        print(f"Generated MeasurementValue IDs (e.g., '{oct_data_with_groups['MeasurementID'].iloc[0]}').")

        print("\nDataFrame with new ID columns added:")
        # Display key columns including the new IDs
        display(oct_data_with_groups[[
            'Sample Name', 'SubjectID', 'AnatomyID', 'AssayID', 'MeasurementID', 'MeasurementValue'
        ]].head())

    except Exception as e:
        print(f"ERROR generating IDs: {e}")
        # Optional: Clear the potentially partially modified dataframe on error
        # oct_data_with_groups = None
else:
    print("Skipping ID generation because oct_data_with_groups DataFrame is not available.")

Generating unique identifiers...
Extracted Subject IDs (e.g., 'MT51').
Generated Assay IDs (e.g., 'OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading').
Generated MeasurementValue IDs (e.g., 'OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading_Nasal_Total_Retinal_Thickness').

DataFrame with new ID columns added:


,Sample Name,SubjectID,AnatomyID,AssayID,MeasurementID,MeasurementValue
0,MT51_OD,MT51,UBERON:0016823,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading_...,0.228
1,MT56_OD,MT56,UBERON:0016823,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading_...,0.241
2,MT59_OD,MT59,UBERON:0016823,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading_...,0.211
3,MT51_OS,MT51,UBERON:0016823,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading_...,0.22
4,MT56_OS,MT56,UBERON:0016823,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading_...,0.224


In [11]:
# --- Create Output Node DataFrames ---
print("Preparing node dataframes for Neo4j import...")
# Check if the main dataframe exists
if 'oct_data_with_groups' in locals() and oct_data_with_groups is not None:
    try:
        # --- Subject Nodes ---
        print("Creating Subject nodes...")
        # Select the SubjectID column, drop duplicates
        subject_nodes = oct_data_with_groups[['SubjectID']].copy().drop_duplicates()
        # Rename SubjectID to the required Neo4j :ID header
        subject_nodes.rename(columns={'SubjectID': 'subjectID:ID'}, inplace=True)
        # Add the :LABEL column
        subject_nodes[':LABEL'] = 'Subject'
        # Reorder columns for clarity / Neo4j standard
        subject_nodes = subject_nodes[['subjectID:ID', ':LABEL']]
        print(f"Created Subject node dataframe: {subject_nodes.shape[0]} unique subjects.")
        display(subject_nodes.head())

        # --- Anatomy Nodes (Proxy Nodes - require ONLY the ID) ---
        print("\nCreating Anatomy nodes...")
        # Select the AnatomyID (UBERON IDs), drop duplicates
        anatomy_nodes = oct_data_with_groups[['AnatomyID']].copy().drop_duplicates()
        # Rename AnatomyID to identifier:ID (using 'identifier' as the property key, which becomes the ID)
        anatomy_nodes.rename(columns={'AnatomyID': 'identifier:ID'}, inplace=True)
         # Add the :LABEL column
        anatomy_nodes[':LABEL'] = 'Anatomy'
        # Reorder columns - IMPORTANT: Proxy nodes need ONLY the identifier:ID column according to hints?
        # Let's keep LABEL for now, might need adjustment based on exact import script behavior
        anatomy_nodes = anatomy_nodes[['identifier:ID', ':LABEL']]
        print(f"Created Anatomy node dataframe: {anatomy_nodes.shape[0]} unique anatomical parts.")
        display(anatomy_nodes.head())

        # --- Assay Nodes ---
        print("\nCreating Assay nodes...")
        # Select relevant columns for Assay properties, drop duplicates based on the unique AssayID
        assay_nodes = oct_data_with_groups[['AssayID', 'MeasurementType', 'Timepoint', 'Sample Name']].copy().drop_duplicates(subset=['AssayID'])
        assay_nodes.rename(columns={
            'AssayID': 'identifier:ID', # Use AssayID as the unique identifier property
            'MeasurementType': 'measurement:string', # Match schema, add type hint
            'Timepoint': 'timepoint:string' # Add our new property with type hint
            }, inplace=True)
        assay_nodes[':LABEL'] = 'Assay' # Assign label
        assay_nodes['technology:string'] = 'Optical Coherence Tomography' # Set fixed value
        # Construct a name (can be customized)
        assay_nodes['name:string'] = assay_nodes['measurement:string'] + ' assay for ' + assay_nodes['Sample Name']

        # Add material_id_1 based on eye (OD/OS) from Sample Name
        assay_nodes['Eye'] = assay_nodes['Sample Name'].str.extract(r'_(OD|OS)')[0]
        eye_map = {'OD': 'UBERON:0004549', 'OS': 'UBERON:0004548'}
        eye_name_map = {'OD': 'Right Eye', 'OS': 'Left Eye'}
        assay_nodes['material_id_1:string'] = assay_nodes['Eye'].map(eye_map)
        assay_nodes['material_name_1:string'] = assay_nodes['Eye'].map(eye_name_map) # Add name too

        # Define the final column order, matching Assay.csv where possible + Neo4j requirements
        final_assay_columns = [
            'identifier:ID',
            ':LABEL',
            'name:string',
            'technology:string',
            'measurement:string',
            'timepoint:string', # Our added property
            'material_id_1:string',
            'material_name_1:string' # Added name for clarity
            # Add other columns like material_1 if needed based on Assay.csv
        ]
        # Select only the columns we defined, in the correct order
        assay_nodes = assay_nodes[[col for col in final_assay_columns if col in assay_nodes.columns]]
        print(f"Created Assay node dataframe: {assay_nodes.shape[0]} unique assays.")
        display(assay_nodes.head())

        # --- MeasurementValue Nodes ---
        print("\nCreating MeasurementValue nodes...")
        # Select relevant columns, drop duplicates based on the unique MeasurementID
        measurement_nodes = oct_data_with_groups[['MeasurementID', 'MeasurementValue', 'Unit', 'MeasurementType']].copy().drop_duplicates(subset=['MeasurementID'])
        measurement_nodes.rename(columns={
            'MeasurementID': 'measurementID:ID', # Use MeasurementID as the unique identifier property
            'MeasurementValue': 'value:float', # Add data type hint
            'MeasurementType': 'type:string',
            'Unit': 'unit:string'
            }, inplace=True)
        measurement_nodes[':LABEL'] = 'MeasurementValue' # Assign label
         # Reorder columns
        measurement_nodes = measurement_nodes[['measurementID:ID', ':LABEL', 'value:float', 'unit:string', 'type:string']]
        print(f"Created MeasurementValue node dataframe: {measurement_nodes.shape[0]} unique measurements.")
        display(measurement_nodes.head())

    except Exception as e:
        print(f"ERROR creating node dataframes: {e}")
        # Clear potentially incomplete dataframes on error
        subject_nodes = None
        anatomy_nodes = None
        assay_nodes = None
        measurement_nodes = None
else:
    print("Skipping node dataframe creation because oct_data_with_groups is not available.")
    subject_nodes = None
    anatomy_nodes = None
    assay_nodes = None
    measurement_nodes = None

Preparing node dataframes for Neo4j import...
Creating Subject nodes...
Created Subject node dataframe: 245 unique subjects.


,subjectID:ID,:LABEL
0,MT51,Subject
1,MT56,Subject
2,MT59,Subject
6,MT66,Subject
7,MT70,Subject



Creating Anatomy nodes...
Created Anatomy node dataframe: 3 unique anatomical parts.


,identifier:ID,:LABEL
0,UBERON:0016823,Anatomy
1310,UBERON:0016824,Anatomy
2616,UBERON:0001773,Anatomy



Creating Assay nodes...
Created Assay node dataframe: 683 unique assays.


,identifier:ID,:LABEL,name:string,technology:string,measurement:string,timepoint:string,material_id_1:string,material_name_1:string
0,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading,Assay,Total Retinal Thickness assay for MT51_OD,Optical Coherence Tomography,Total Retinal Thickness,End_of_Hindlimb_Unloading,UBERON:0004549,Right Eye
1,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading,Assay,Total Retinal Thickness assay for MT56_OD,Optical Coherence Tomography,Total Retinal Thickness,End_of_Hindlimb_Unloading,UBERON:0004549,Right Eye
2,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading,Assay,Total Retinal Thickness assay for MT59_OD,Optical Coherence Tomography,Total Retinal Thickness,End_of_Hindlimb_Unloading,UBERON:0004549,Right Eye
3,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading,Assay,Total Retinal Thickness assay for MT51_OS,Optical Coherence Tomography,Total Retinal Thickness,End_of_Hindlimb_Unloading,UBERON:0004548,Left Eye
4,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading,Assay,Total Retinal Thickness assay for MT56_OS,Optical Coherence Tomography,Total Retinal Thickness,End_of_Hindlimb_Unloading,UBERON:0004548,Left Eye



Creating MeasurementValue nodes...
Created MeasurementValue node dataframe: 3919 unique measurements.


,measurementID:ID,:LABEL,value:float,unit:string,type:string
0,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading_...,MeasurementValue,0.228,millimeter,Total Retinal Thickness
1,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading_...,MeasurementValue,0.241,millimeter,Total Retinal Thickness
2,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading_...,MeasurementValue,0.211,millimeter,Total Retinal Thickness
3,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading_...,MeasurementValue,0.22,millimeter,Total Retinal Thickness
4,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading_...,MeasurementValue,0.224,millimeter,Total Retinal Thickness


In [12]:
# --- Create Output Relationship DataFrames ---
print("Preparing relationship dataframes for Neo4j import...")
# Check if the main dataframe with IDs exists
if 'oct_data_with_groups' in locals() and oct_data_with_groups is not None:
    try:
        # Relationship 1: Assay PERFORMED_ON Subject
        print("\nCreating Assay-PERFORMED_ON-Subject relationships...")
        # Select the IDs for Assay (start) and Subject (end)
        assay_subject_rels = oct_data_with_groups[['AssayID', 'SubjectID']].copy()
        # Remove duplicates - one Assay is performed on one Subject
        assay_subject_rels.drop_duplicates(inplace=True)
        # Rename columns to Neo4j import headers
        assay_subject_rels.rename(columns={'AssayID': ':START_ID', 'SubjectID': ':END_ID'}, inplace=True)
        # Assign the relationship type
        assay_subject_rels[':TYPE'] = 'PERFORMED_ON'
        print(f"Created Assay-PERFORMED_ON-Subject dataframe: {assay_subject_rels.shape[0]} relationships.")
        display(assay_subject_rels.head())

        # Relationship 2: Assay HAS_OUTPUT MeasurementValue
        print("\nCreating Assay-HAS_OUTPUT-MeasurementValue relationships...")
        # Select the IDs for Assay (start) and MeasurementValue (end)
        assay_measure_rels = oct_data_with_groups[['AssayID', 'MeasurementID']].copy()
        # Drop duplicates (should be unique already if MeasurementID is unique per assay)
        assay_measure_rels.drop_duplicates(inplace=True)
        # Rename columns
        assay_measure_rels.rename(columns={'AssayID': ':START_ID', 'MeasurementID': ':END_ID'}, inplace=True)
        # Assign the relationship type
        assay_measure_rels[':TYPE'] = 'HAS_OUTPUT'
        print(f"Created Assay-HAS_OUTPUT-MeasurementValue dataframe: {assay_measure_rels.shape[0]} relationships.")
        display(assay_measure_rels.head())

        # Relationship 3: MeasurementValue MEASURES_ANATOMY Anatomy
        print("\nCreating MeasurementValue-MEASURES_ANATOMY-Anatomy relationships...")
        # Select the IDs for MeasurementValue (start) and Anatomy (end)
        measure_anatomy_rels = oct_data_with_groups[['MeasurementID', 'AnatomyID']].copy()
         # Drop duplicates (a specific measurement measures one specific anatomy part)
        measure_anatomy_rels.drop_duplicates(inplace=True)
         # Rename columns
        measure_anatomy_rels.rename(columns={'MeasurementID': ':START_ID', 'AnatomyID': ':END_ID'}, inplace=True)
        # Assign the relationship type
        measure_anatomy_rels[':TYPE'] = 'MEASURES_ANATOMY'
        print(f"Created MeasurementValue-MEASURES_ANATOMY-Anatomy dataframe: {measure_anatomy_rels.shape[0]} relationships.")
        display(measure_anatomy_rels.head())

        # --- Add other relationships here if needed based on your model ---
        # e.g., Subject -> Study, Assay -> Study, Anatomy -> Anatomy (part_of)

    except Exception as e:
        print(f"ERROR creating relationship dataframes: {e}")
else:
    print("Skipping relationship dataframe creation because oct_data_with_groups is not available.")

Preparing relationship dataframes for Neo4j import...

Creating Assay-PERFORMED_ON-Subject relationships...
Created Assay-PERFORMED_ON-Subject dataframe: 683 relationships.


,:START_ID,:END_ID,:TYPE
0,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading,MT51,PERFORMED_ON
1,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading,MT56,PERFORMED_ON
2,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading,MT59,PERFORMED_ON
3,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading,MT51,PERFORMED_ON
4,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading,MT56,PERFORMED_ON



Creating Assay-HAS_OUTPUT-MeasurementValue relationships...
Created Assay-HAS_OUTPUT-MeasurementValue dataframe: 3919 relationships.


,:START_ID,:END_ID,:TYPE
0,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading_...,HAS_OUTPUT
1,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading_...,HAS_OUTPUT
2,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading_...,HAS_OUTPUT
3,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading_...,HAS_OUTPUT
4,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading_...,HAS_OUTPUT



Creating MeasurementValue-MEASURES_ANATOMY-Anatomy relationships...
Created MeasurementValue-MEASURES_ANATOMY-Anatomy dataframe: 3919 relationships.


,:START_ID,:END_ID,:TYPE
0,OSD-679_OCT_MT51_OD_End_of_Hindlimb_Unloading_...,UBERON:0016823,MEASURES_ANATOMY
1,OSD-679_OCT_MT56_OD_End_of_Hindlimb_Unloading_...,UBERON:0016823,MEASURES_ANATOMY
2,OSD-679_OCT_MT59_OD_End_of_Hindlimb_Unloading_...,UBERON:0016823,MEASURES_ANATOMY
3,OSD-679_OCT_MT51_OS_End_of_Hindlimb_Unloading_...,UBERON:0016823,MEASURES_ANATOMY
4,OSD-679_OCT_MT56_OS_End_of_Hindlimb_Unloading_...,UBERON:0016823,MEASURES_ANATOMY


In [13]:
# --- Write Output CSV Files for Neo4j Import ---
import csv # Required for quoting options in to_csv

print("Preparing to write output CSV files...")

# Define the DataFrames we want to save and their desired filenames
# Make sure the keys match the variable names of your final DataFrames
dataframes_to_save = {
    'subject_nodes': 'Subject_nodes.csv',
    'anatomy_nodes': 'Anatomy_nodes.csv',
    'assay_nodes': 'Assay_nodes.csv',
    'measurement_nodes': 'MeasurementValue_nodes.csv',
    'assay_subject_rels': 'Assay_PERFORMED_ON_Subject_rels.csv',
    'assay_measure_rels': 'Assay_HAS_OUTPUT_MeasurementValue_rels.csv',
    'measure_anatomy_rels': 'MeasurementValue_MEASURES_ANATOMY_Anatomy_rels.csv'
    # Add any other node or relationship dataframes you created here
    # e.g., 'subject_study_rels': 'Subject_PART_OF_STUDY_Study_rels.csv'
}

# Check if all prerequisite DataFrames actually exist before proceeding
all_dfs_exist = True
for df_name in dataframes_to_save.keys():
    if not (df_name in locals() and isinstance(locals()[df_name], pd.DataFrame)):
        print(f"ERROR: DataFrame variable '{df_name}' is missing or not a DataFrame. Cannot save files.")
        all_dfs_exist = False
        break # Stop if any required DataFrame is missing

if all_dfs_exist:
    try:
        # Define the output directory path (e.g., create 'output_csvs' inside project dir)
        # You can change this path if you prefer
        output_dir = os.path.join(kg_git_dir, 'output_csvs')
        os.makedirs(output_dir, exist_ok=True) # Create the directory if it doesn't exist
        print(f"Output directory set to: {output_dir}")

        # Loop through the dictionary and save each DataFrame to a CSV file
        for df_variable_name, output_filename in dataframes_to_save.items():
            # Get the actual DataFrame variable from its name string
            df_to_save = locals()[df_variable_name]
            output_path = os.path.join(output_dir, output_filename)

            print(f"Saving {output_filename} ({len(df_to_save)} rows)...")
            # Save to CSV using recommended Neo4j import settings
            df_to_save.to_csv(
                output_path,
                index=False,       # Do not write the pandas DataFrame index
                header=True,       # Write the header row (containing :ID, :LABEL, etc.)
                sep=',',           # Use comma as the separator
                quoting=csv.QUOTE_MINIMAL # Quote fields only if they contain the separator or quote char
            )
        print("\nAll output CSV files for OCT data saved successfully to the 'output_csvs' folder!")
        print("Objective 1 (Data Readiness) for OCT in OSD-679 is effectively complete.")

    except Exception as e:
        print(f"ERROR writing CSV files: {e}")
else:
    print("Skipping CSV writing due to missing prerequisite DataFrames.")

Preparing to write output CSV files...
Output directory set to: /Users/gastondana/Desktop/spoke_genelab/output_csvs
Saving Subject_nodes.csv (245 rows)...
Saving Anatomy_nodes.csv (3 rows)...
Saving Assay_nodes.csv (683 rows)...
Saving MeasurementValue_nodes.csv (3919 rows)...
Saving Assay_PERFORMED_ON_Subject_rels.csv (683 rows)...
Saving Assay_HAS_OUTPUT_MeasurementValue_rels.csv (3919 rows)...
Saving MeasurementValue_MEASURES_ANATOMY_Anatomy_rels.csv (3919 rows)...

All output CSV files for OCT data saved successfully to the 'output_csvs' folder!
Objective 1 (Data Readiness) for OCT in OSD-679 is effectively complete.
